Assignment - 2

Word Count

In [1]:
import findspark
findspark.init()
from pyspark.sql.functions import explode, split, col, desc
from pyspark.sql import SparkSession
import nltk
from nltk.corpus import stopwords
import shutil
import os
import string

#nltk.download('stopwords')
stop_words = set(stopwords.words('english')) 

output_path_1 = "output_1"

# Remove the output directories if they already exist
if os.path.exists(output_path_1):
    shutil.rmtree(output_path_1)

try:
    spark = SparkSession.builder.appName("WordCountApp").master("local[*]").getOrCreate()
    sc = spark.sparkContext
except Exception as e:
    print(f"An error occurred: {e}")


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/10 17:57:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
try:
    text_file = sc.textFile("littlewoman.txt")
    text_file_2 = sc.textFile("pride_and_prejudice.txt")
    
    count_combined = text_file.union(text_file_2)
    count_combined_transformation = count_combined.flatMap(lambda line: line.translate(str.maketrans("", "", string.punctuation)).lower().split())
    count_combined_filter= count_combined_transformation.filter(lambda word: word not in stop_words)
    count_combined_mapping = count_combined_filter.map(lambda word: (word, 1))
    count_combined_unique = count_combined_mapping.reduceByKey(lambda a, b: a + b)
    count_combined_sorted = count_combined_unique.sortBy(lambda x: x[1],ascending=False)

    count_combined_sorted.saveAsTextFile(output_path_1)

    # Merge partition files into a single output file

    with open("output_1.txt", "w") as outfile:
        for filename in sorted(os.listdir(output_path_1)):
            if filename.startswith("part-"):
                with open(os.path.join(output_path_1, filename), "r") as infile:
                    outfile.write(infile.read())
    
    text_df = spark.read.text("output_1.txt")

    top_25_words = text_df.limit(25)
    top_25_words.show(25, truncate=False)

except Exception as e:
     print(f"An error occurred: {e}")

    

+------------------+
|value             |
+------------------+
|('jo', 1293)      |
|('said', 1245)    |
|('one', 1159)     |
|('mr', 1123)      |
|('little', 961)   |
|('would', 929)    |
|('could', 893)    |
|('much', 704)     |
|('like', 676)     |
|('meg', 653)      |
|('mrs', 606)      |
|('never', 605)    |
|('elizabeth', 601)|
|('amy', 588)      |
|('see', 574)      |
|('good', 572)     |
|('laurie', 564)   |
|('well', 557)     |
|('know', 557)     |
|('dont', 552)     |
|('time', 522)     |
|('go', 501)       |
|('think', 496)    |
|('must', 462)     |
|('away', 453)     |
+------------------+



In [3]:
sc.stop()

In [5]:
import re
from pyspark.sql import SparkSession
import findspark
findspark.init()


# Initialize Spark session
spark = SparkSession.builder.appName("PageRank").master("local[*]").getOrCreate()
sc = spark.sparkContext

# Load and parse the network file
lines = sc.textFile("question3.txt")

# Parse each line into (page, neighbors) pairs
def parse_neighbors(line):
    parts = re.split(r':\s*\[|\]', line)
    if len(parts) < 2:
        return None
    page = parts[0].strip()
    neighbors = parts[1].strip().split(', ')
    return page, neighbors

# Create an RDD of (page, list of neighbors)
links = lines.map(parse_neighbors).filter(lambda x: x is not None)

# Initialize each page's rank to 1.0
ranks = links.mapValues(lambda _: 1.0)

# Number of iterations for convergence
iterations = 10
damping_factor = 0.85  # Damping factor for PageRank

# Run PageRank algorithm for a fixed number of iterations
for _ in range(iterations):
    # Calculate contributions for each page
    contributions = links.join(ranks).flatMap(
        lambda page_neighbors_rank: [(neighbor, page_neighbors_rank[1][1] / len(page_neighbors_rank[1][0])) 
                                     for neighbor in page_neighbors_rank[1][0]]
    )
    
    # Calculate new ranks based on contributions
    ranks = contributions.reduceByKey(lambda a, b: a + b).mapValues(
        lambda rank: (1 - damping_factor) + damping_factor * rank
    )

# Collect and save the final ranks to output file
page_ranks = ranks.collect()
with open("output_page_ranks.txt", "w") as f:
    for page, rank in page_ranks:
        f.write(f"{page}: {rank}\n")

# Find the page with the highest and lowest PageRank
max_page = max(page_ranks, key=lambda x: x[1])
min_page = min(page_ranks, key=lambda x: x[1])

print(f"Page with highest rank: {max_page}")
print(f"Page with lowest rank: {min_page}")



24/11/10 17:58:36 WARN SparkContext: Another SparkContext is being constructed (or threw an exception in its constructor). This may indicate an error, since only one SparkContext should be running in this JVM (see SPARK-2243). The other SparkContext was created at:
org.apache.spark.api.java.JavaSparkContext.<init>(JavaSparkContext.scala:58)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
java.base/jdk.internal.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImpl.java:62)
java.base/jdk.internal.reflect.DelegatingConstructorAccessorImpl.newInstance(DelegatingConstructorAccessorImpl.java:45)
java.base/java.lang.reflect.Constructor.newInstance(Constructor.java:490)
py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:247)
py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:374)
py4j.Gateway.invoke(Gateway.java:238)
py4j.commands.ConstructorCommand.invokeConstructor(ConstructorCommand.java:80)
py4j.commands.Con

Page with highest rank: ('50', 4.176179975971073)
Page with lowest rank: ('35', 0.2593061945644343)


Djikstra

In [4]:
from pyspark import SparkContext, SparkConf
import findspark
findspark.init()
from pyspark.sql.functions import explode, split, col, desc
from pyspark.sql import SparkSession

# Initialize SparkContext
spark = SparkSession.builder.appName("Djikstra").master("local[*]").getOrCreate()
sc = spark.sparkContext


# Read the data files
edges_rdd = sc.textFile('question2_1.txt').union(sc.textFile('question2_2.txt'))

# Parse the edges into (source_node, destination_node, weight)
edges = edges_rdd.map(lambda line: line.strip().split(',')) \
                 .map(lambda parts: (int(parts[0]), int(parts[1]), float(parts[2])))

# Create an adjacency list RDD: (node, list of (neighbor, weight))
adjacency_list = edges.map(lambda x: (x[0], (x[1], x[2]))) \
                      .groupByKey() \
                      .mapValues(list) \
                      .cache()

# Get all nodes
nodes_from = edges.map(lambda x: x[0])
nodes_to = edges.map(lambda x: x[1])
all_nodes = nodes_from.union(nodes_to).distinct().cache()

# Initialize distances: (node, distance)
start_node = 0  # Assuming the first node is 0
infinity = float('inf')
distances = all_nodes.map(lambda node: (node, infinity))
distances = distances.map(lambda x: (x[0], 0.0) if x[0] == start_node else x)
distances = distances.cache()

# Iterative update of distances
updated = True
iteration = 0
max_iterations = all_nodes.count() - 1  # Maximum possible iterations

while updated and iteration < max_iterations:
    iteration += 1
    # Join distances with adjacency list
    joined = distances.join(adjacency_list, numPartitions=8)
    
    # Compute tentative distances
    tentative_distances = joined.flatMap(lambda x: [ 
        (neighbor[0], x[1][0] + neighbor[1]) for neighbor in x[1][1]
    ])
    
    # Combine the new distances with the existing ones
    new_distances = distances.union(tentative_distances) \
                             .reduceByKey(lambda x, y: min(x, y))
    
    # Check if distances have changed
    changes = new_distances.join(distances).filter(lambda x: x[1][0] != x[1][1])
    updated = not changes.isEmpty()
    
    # Update distances for the next iteration
    distances = new_distances
    distances = distances.cache()

# Collect final distances
final_distances = distances.collectAsMap()

# Write distances to output file
with open('output_2.txt', 'w') as f:
    for node in sorted(final_distances.keys()):
        dist = final_distances[node]
        if dist == infinity:
            f.write(f"{node} unreachable\n")
        else:
            f.write(f"{node} {dist}\n")

# Find nodes with greatest and least distances (excluding infinity and the start node)
reachable_nodes = {node: dist for node, dist in final_distances.items() if dist != infinity and node != start_node}
if reachable_nodes:
    # Find the maximum and minimum distances
    max_distance = max(reachable_nodes.values())
    min_distance = min(reachable_nodes.values())
    
    # Find all nodes that have the maximum and minimum distances
    max_nodes = [node for node, dist in reachable_nodes.items() if dist == max_distance]
    min_nodes = [node for node, dist in reachable_nodes.items() if dist == min_distance]
    
    # Print nodes with greatest distance
    print(f"Nodes with greatest distance from {start_node} (Distance: {max_distance}): {max_nodes}")
    
    # Print nodes with least distance
    print(f"Nodes with least distance from {start_node} (Distance: {min_distance}): {min_nodes}")
else:
    print("No reachable nodes from the starting node.")




Nodes with greatest distance from 0 (Distance: 3.0): [32, 2, 35, 11, 13, 15, 51, 20, 90]
Nodes with least distance from 0 (Distance: 1.0): [1, 66, 6, 7, 39, 71, 40, 9, 41, 75, 43, 76, 14, 16, 49, 19, 53, 54, 87, 57, 28, 60, 92]


In [5]:
sc.stop()

Page Rank